In [1]:
#import findspark
#findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import os

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1575409539261_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/73/9b/52e228545d14f14bb2a1622e225f38463c8726645165e1cb7dde95bfe6d4/pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

In [5]:
from pyspark.sql.functions import monotonically_increasing_id 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Import 

In [6]:
import numpy as np
import pandas as pd

from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.feature import IDF

from pyspark.ml import Pipeline, PipelineModel


from pyspark.sql.functions import *

from pyspark.sql.types import *
from pyspark import SparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql.types import *

from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Word2Vec, Word2VecModel

from pyspark.ml.clustering import LDA, LDAModel, LocalLDAModel
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
#from wordcloud import WordCloud

from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
sc = spark.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
hotel_df_temp = spark.read.csv( 's3://aws-emr-resources-140484683578-us-east-1/notebooks/e-2AFROLBBL6SA34XDT9KVU1RAE/Datafiniti_Hotel_Reviews.csv', header=True)
# print the schema of the review dataframe
hotel_df_temp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- dateAdded: string (nullable = true)
 |-- dateUpdated: string (nullable = true)
 |-- address: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- primaryCategories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- name: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- province: string (nullable = true)
 |-- reviews.date: string (nullable = true)
 |-- reviews.dateSeen: string (nullable = true)
 |-- reviews.rating: string (nullable = true)
 |-- reviews.sourceURLs: string (nullable = true)
 |-- reviews.text: string (nullable = true)
 |-- reviews.title: string (nullable = true)
 |-- reviews.userCity: string (nullable = true)
 |-- reviews.userProvince: string (nullable = true)
 |-- reviews.username: string (nullable = true)
 |-- sourceU

## Rename Columns

In [11]:
def rename(df, old_columns, new_columns):
    for old_col,new_col in zip(old_columns,new_columns):
        df = df.withColumnRenamed(old_col,new_col)
    return df
oldColumns = hotel_df_temp.schema.names
print(oldColumns)
newColumns = ["hotel_id", "dateAdded","dateUpdated","address","categories","primaryCategories","city","country","keys","latitude","longitude","name","postalCode","province","reviews_date","reviews_dateSeen","reviews_rating","reviews_sourceURLs","reviews_text","reviews_title","reviews_userCity","reviews_userProvince","reviews_username","sourceURLs","websites"]
hotel_df_original = rename(hotel_df_temp, oldColumns, newColumns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['id', 'dateAdded', 'dateUpdated', 'address', 'categories', 'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude', 'name', 'postalCode', 'province', 'reviews.date', 'reviews.dateSeen', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.userCity', 'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites']

## Replace NaN values

In [12]:
hotel_df_original = hotel_df_original.filter(hotel_df_original.reviews_username.isNotNull())
print(hotel_df_original.count())
colNames = ['reviews_userCity','reviews_userProvince']
hotel_df_original = hotel_df_original.na.fill("Not given", colNames)

hotel_df_original.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9998
root
 |-- hotel_id: string (nullable = true)
 |-- dateAdded: string (nullable = true)
 |-- dateUpdated: string (nullable = true)
 |-- address: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- primaryCategories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- name: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- province: string (nullable = true)
 |-- reviews_date: string (nullable = true)
 |-- reviews_dateSeen: string (nullable = true)
 |-- reviews_rating: string (nullable = true)
 |-- reviews_sourceURLs: string (nullable = true)
 |-- reviews_text: string (nullable = true)
 |-- reviews_title: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)
 |-- reviews_username: string (nullable = true)

## Exploratory Analysis

In [13]:
hotel_df = hotel_df_original.select('hotel_id', 'name','address','categories','primaryCategories','city','country','keys','latitude','longitude','postalCode','province').persist()

hotel_df =hotel_df.dropDuplicates(['hotel_id'])

hotel_df.printSchema()
print(hotel_df.count())
hotel_df.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hotel_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- primaryCategories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- province: string (nullable = true)

1853
1853

In [14]:
user_df = hotel_df_original.select( 'reviews_username','reviews_userCity','reviews_userProvince')

user_df =user_df.dropDuplicates(['reviews_username','reviews_userCity','reviews_userProvince'])
print(user_df.count())
user_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7234
root
 |-- reviews_username: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)

In [15]:
from pyspark.sql.window import Window  
w = Window.orderBy("hotel_id") 
hotel_df_original = hotel_df_original.withColumn("index", row_number().over(w))
hotel_df_original.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hotel_id: string (nullable = true)
 |-- dateAdded: string (nullable = true)
 |-- dateUpdated: string (nullable = true)
 |-- address: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- primaryCategories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- name: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- province: string (nullable = true)
 |-- reviews_date: string (nullable = true)
 |-- reviews_dateSeen: string (nullable = true)
 |-- reviews_rating: string (nullable = true)
 |-- reviews_sourceURLs: string (nullable = true)
 |-- reviews_text: string (nullable = true)
 |-- reviews_title: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)
 |-- reviews_username: string (nullable = true)
 |--

In [16]:
review_df = hotel_df_original.select( 'index','hotel_id','reviews_username','reviews_date','reviews_dateSeen','reviews_rating','reviews_sourceURLs','reviews_text','reviews_title','reviews_userCity','reviews_userProvince')
review_df.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- index: integer (nullable = true)
 |-- hotel_id: string (nullable = true)
 |-- reviews_username: string (nullable = true)
 |-- reviews_date: string (nullable = true)
 |-- reviews_dateSeen: string (nullable = true)
 |-- reviews_rating: string (nullable = true)
 |-- reviews_sourceURLs: string (nullable = true)
 |-- reviews_text: string (nullable = true)
 |-- reviews_title: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)

In [18]:
w = Window.orderBy('reviews_username','reviews_userCity','reviews_userProvince')
user_new_df = user_df.withColumn("uid", row_number().over(w))

user_new_df.printSchema()
user_new_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- reviews_username: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)
 |-- uid: integer (nullable = true)

7234

In [19]:

user_new_df.printSchema()
user_new_df.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- reviews_username: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)
 |-- uid: integer (nullable = true)

7234

In [20]:
hotel_df_original.registerTempTable("temp")
df2 = sqlContext.sql("select hotel_id,avg(reviews_rating) as average_rating from temp group by hotel_id")
df2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hotel_id: string (nullable = true)
 |-- average_rating: double (nullable = true)

In [21]:
a = hotel_df.alias("a")
b = df2.alias("b")
  
hotel_df_with_avgRating = a.join(b, col("a.hotel_id") == col("b.hotel_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.average_rating')])

#business_df.show(2)
hotel_df_with_avgRating.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hotel_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- primaryCategories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- province: string (nullable = true)
 |-- average_rating: double (nullable = true)

In [22]:
hotel_newid_df = sqlContext.createDataFrame(hotel_df.rdd.map(lambda x: x[0]).zipWithIndex(), \
        StructType([StructField("hotel_id", StringType(), True),StructField("Id", IntegerType(), True)])).persist()

#business_newid_df.distinct().count()
hotel_newid_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hotel_id: string (nullable = true)
 |-- Id: integer (nullable = true)

In [23]:
a = hotel_df_with_avgRating.alias("a")
a.show()
b = hotel_newid_df.alias("b")
    
hotel_new_df = a.join(b, col("a.hotel_id") == col("b.hotel_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.Id')])

hotel_new_df.select('Id','hotel_id', 'name').show(2)
hotel_new_df.printSchema()
hotel_new_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------+--------------------+---------+-----------+----------+--------+-----------------+
|            hotel_id|                name|             address|          categories|   primaryCategories|         city|country|                keys| latitude|  longitude|postalCode|province|   average_rating|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------+--------------------+---------+-----------+----------+--------+-----------------+
|AVweOoAPkufWRAb5BHJO|Portland Regency ...|          20 Milk St|Hotels,Bar,Lounge...|Accommodation & F...|     Portland|     US|us/me/portland/20...|  43.6569|  -70.25256|      4101|      ME|4.444444444444445|
|AVwcgvkEkufWRAb5v9BO|    Super 8 St Cloud|       50 Park Ave S|Lodging,Motels,Ho...|Accommodation & F...|  Saint Cloud|     US|us/mn/saintcloud/...| 45.55158| 

In [24]:
review_df.printSchema()
print(review_df.distinct().count())
review_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- index: integer (nullable = true)
 |-- hotel_id: string (nullable = true)
 |-- reviews_username: string (nullable = true)
 |-- reviews_date: string (nullable = true)
 |-- reviews_dateSeen: string (nullable = true)
 |-- reviews_rating: string (nullable = true)
 |-- reviews_sourceURLs: string (nullable = true)
 |-- reviews_text: string (nullable = true)
 |-- reviews_title: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)

9998
+-----+--------------------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+----------------+--------------------+
|index|            hotel_id|reviews_username|        reviews_date|    reviews_dateSeen|reviews_rating|  reviews_sourceURLs|        reviews_text|       reviews_title|reviews_userCity|reviews_userProvince|
+-----+--------------------+----------------+--------------------+----------

In [25]:
user_new_df.printSchema()
print(user_new_df.distinct().count())
print(user_new_df.count())

user_new_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- reviews_username: string (nullable = true)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_userProvince: string (nullable = false)
 |-- uid: integer (nullable = true)

7234
7234
+-----------------+--------------------+--------------------+---+
| reviews_username|    reviews_userCity|reviews_userProvince|uid|
+-----------------+--------------------+--------------------+---+
| die Lage super."|      dasBett bequem| das Personal fre...|  1|
|        soap scum| etc. - very unsa...| leaving shaving ...|  2|
|    'Sina Bamtefa|           Not given|           Not given|  3|
|          007lele|            Edmonton|           Not given|  4|
|         0501MVKG|         Spring Hill|                  TN|  5|
|          0ls0njo|                Lodi|                  WI|  6|
|         0theHero|           Not given|           Not given|  7|
|       103bennier|         West Branch|                  MI|  8|
|       106PamelaL|               Moody|           Not given|  9|
|     

In [26]:
review_df = review_df.select('index','reviews_userProvince','reviews_userCity','reviews_username', 'hotel_id', 'reviews_rating')


# userid match
a = review_df.alias("a")

b = user_new_df.alias("b")
    
review_userId_df = a.join(b, (col("a.reviews_username") == col("b.reviews_username")) & (col("a.reviews_userCity") == col("b.reviews_userCity")) & (col("a.reviews_userProvince") == col("b.reviews_userProvince")), 'inner') \
                     .select([col('a.'+xx) for xx in a.columns] + [col('b.uid')]).persist()

# categories match
review_userId_df.printSchema()
print(review_userId_df.distinct().count())
print(review_userId_df.count())
review_userId_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- index: integer (nullable = true)
 |-- reviews_userProvince: string (nullable = false)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_username: string (nullable = true)
 |-- hotel_id: string (nullable = true)
 |-- reviews_rating: string (nullable = true)
 |-- uid: integer (nullable = true)

9998
9998
+-----+--------------------+----------------+----------------+--------------------+--------------+----+
|index|reviews_userProvince|reviews_userCity|reviews_username|            hotel_id|reviews_rating| uid|
+-----+--------------------+----------------+----------------+--------------------+--------------+----+
|    1|           Not given|       Not given|            Lori|AV-TGsFqRxPSIh2RmVF-|           4.6|3184|
|    2|           Not given|       Not given|         Rohanna|AV-TGsFqRxPSIh2RmVF-|           4.8|4395|
|    3|           Not given|       Not given|           Maria|AV-TGsFqRxPSIh2RmVF-|          4.15|3351|
|    4|           Not given|       Not given|     

In [27]:
df_filtered_review=review_userId_df.filter(review_userId_df.reviews_username=='Lori')
df_filtered_review.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+----------------+----------------+--------------------+--------------+----+
|index|reviews_userProvince|reviews_userCity|reviews_username|            hotel_id|reviews_rating| uid|
+-----+--------------------+----------------+----------------+--------------------+--------------+----+
|    1|           Not given|       Not given|            Lori|AV-TGsFqRxPSIh2RmVF-|           4.6|3184|
|  129|           Not given|       Not given|            Lori|AV-TGwK3IxWefVJwr5IJ|             5|3184|
|  499|           Not given|       Not given|            Lori|AV1ha6JpIxWefVJwgDSa|             5|3184|
+-----+--------------------+----------------+----------------+--------------------+--------------+----+

In [28]:
a = review_userId_df.alias("a")
b = hotel_newid_df.alias("b")

review_userId_hotelId_df = a.join(b, col("a.hotel_id") == col("b.hotel_id"), 'inner') \
                         .select([col('a.'+xx) for xx in a.columns]+ [col('b.Id')]).persist() 

review_userId_hotelId_df.printSchema()
review_userId_hotelId_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- index: integer (nullable = true)
 |-- reviews_userProvince: string (nullable = false)
 |-- reviews_userCity: string (nullable = false)
 |-- reviews_username: string (nullable = true)
 |-- hotel_id: string (nullable = true)
 |-- reviews_rating: string (nullable = true)
 |-- uid: integer (nullable = true)
 |-- Id: integer (nullable = true)

9998

In [29]:
df_filtered_review=review_userId_hotelId_df.filter(review_userId_hotelId_df.reviews_username=='Lori')
df_filtered_review.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+----------------+----------------+--------------------+--------------+----+----+
|index|reviews_userProvince|reviews_userCity|reviews_username|            hotel_id|reviews_rating| uid|  Id|
+-----+--------------------+----------------+----------------+--------------------+--------------+----+----+
|    1|           Not given|       Not given|            Lori|AV-TGsFqRxPSIh2RmVF-|           4.6|3184| 800|
|  129|           Not given|       Not given|            Lori|AV-TGwK3IxWefVJwr5IJ|             5|3184|1443|
|  499|           Not given|       Not given|            Lori|AV1ha6JpIxWefVJwgDSa|             5|3184|1046|
+-----+--------------------+----------------+----------------+--------------------+--------------+----+----+

In [30]:
from pyspark.sql.functions import isnan, when, count, col

review_userId_hotelId_df.select([count(when(isnan(c), c)).alias(c) for c in review_userId_hotelId_df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+----------------+----------------+--------+--------------+---+---+
|index|reviews_userProvince|reviews_userCity|reviews_username|hotel_id|reviews_rating|uid| Id|
+-----+--------------------+----------------+----------------+--------+--------------+---+---+
|    0|                   0|               0|               0|       0|             0|  0|  0|
+-----+--------------------+----------------+----------------+--------+--------------+---+---+

In [31]:
#review_userId_businessId_df.distinct().count()
review_userId_hotelId_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+----------------+----------------+--------------------+--------------+----+---+
|index|reviews_userProvince|reviews_userCity|reviews_username|            hotel_id|reviews_rating| uid| Id|
+-----+--------------------+----------------+----------------+--------------------+--------------+----+---+
|    1|           Not given|       Not given|            Lori|AV-TGsFqRxPSIh2RmVF-|           4.6|3184|800|
|    2|           Not given|       Not given|         Rohanna|AV-TGsFqRxPSIh2RmVF-|           4.8|4395|800|
|    3|           Not given|       Not given|           Maria|AV-TGsFqRxPSIh2RmVF-|          4.15|3351|800|
|    4|           Not given|       Not given|            Rami|AV-TGsFqRxPSIh2RmVF-|           2.5|4176|800|
|    5|           Not given|       Not given|         Matthew|AV-TGsFqRxPSIh2RmVF-|          3.95|3468|800|
|    6|           Not given|       Not given|             mxk|AV-TGsFqRxPSIh2RmVF-|           4.6|6610|800|
|    7|           Not given|

In [33]:
rating_df = review_userId_hotelId_df.select('uid', 'Id', review_userId_hotelId_df.reviews_rating.cast('float').alias('rating')).persist()

rating_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- uid: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- rating: float (nullable = true)

In [34]:

rating_df.limit(20).toPandas()
from pyspark.sql.functions import isnan, when, count, col

rating_df.select([count(when(isnan(c), c)).alias(c) for c in rating_df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+------+
|uid| Id|rating|
+---+---+------+
|  0|  0|     0|
+---+---+------+

In [35]:
(train, test) = rating_df.randomSplit([0.8, 0.2], seed=123)
rating_df.select("uid").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7234

## ALS model

In [36]:
import warnings
warnings.filterwarnings('ignore')

als = ALS(userCol="uid", itemCol="Id", ratingCol="rating", coldStartStrategy="drop")

param_grid = ParamGridBuilder().addGrid(
    als.rank,
    [10, 15,20],
).addGrid(
    als.maxIter,
    [10, 15,20],
).build()

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
)

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5, seed=123)
cv_als_model = cv.fit(train)

# Evaluate the model by computing the RMSE on the test data

als_predictions = cv_als_model.bestModel.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(als_predictions)
print("Root-mean-square error = " + str(rmse))

best_model = cv_als_model.bestModel

#Optimal rank is 20
best_rank = best_model.rank
print(best_rank)
# Optimal maxIter is 20
best_getMaxIter = (best_model._java_obj.parent().getMaxIter()) # Get maxIter


# Root-mean-square error is  1.6881326329174966

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-36:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1148



Root-mean-square error = 1.6881326329174966
20

In [37]:
alsb = ALS(rank=best_rank, maxIter=best_getMaxIter, regParam=0.3, userCol="uid", itemCol="Id", ratingCol="rating", \
               coldStartStrategy="drop", seed=123)
alsb_model = alsb.fit(train)

alsb_predictions = alsb_model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(alsb_predictions)
print("Root-mean-square error = " + str(rmse))

# ALS model


# Root-mean-square error is 1.6216368297163042

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root-mean-square error = 1.6216368297163042

In [39]:
userRecoms = alsb_model.recommendForAllUsers(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"userRecoms.orderBy('uid').show(truncate=False)\ndf_filtered_review=user_new_df.filter(user_new_df.uid==6)\ndf_filtered_review.show()\nuserRecoms.count()"

In [40]:
a = userRecoms.alias("a")
b = user_new_df.alias("b")
    
all_userRecoms = a.join(b, col("a.uid") == col("b.uid"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.reviews_username')]).persist()

all_userRecoms.cache()   
all_userRecoms.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-37:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 5859



5806

In [41]:
user_new_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7234

In [42]:
import warnings
warnings.filterwarnings('ignore')
u_id = 6

userFlatRec =  sqlContext.createDataFrame(all_userRecoms.filter(col('uid') == u_id).rdd.flatMap(lambda p: p[1])).persist()
userFlatRec.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: long (nullable = true)
 |-- rating: double (nullable = true)

Exception in thread cell_monitor-42:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 6062



In [43]:
a = hotel_new_df.alias("a")
b = userFlatRec.alias("b")

user_collab_df = a.join(b, col("a.Id") == col("b.Id"), 'inner') \
                         .select([col('a.'+xx) for xx in a.columns] + [col('b.rating')])
user_collab_df.printSchema()
    
user_collab_df.select('hotel_id', 'name', 'rating', 'categories').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hotel_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- primaryCategories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- province: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- Id: integer (nullable = true)
 |-- rating: double (nullable = true)

               hotel_id  ...                                         categories
0  AVwdygbvIN2L1WUfzOvF  ...                        Hotels,Resort,Lodging,Hotel
1  AVwdEH8rByjofQCxlJBS  ...  Hotels,Boutique Hotels,Tourist Hotels,Corporat...
2  AVwct0d4IN2L1WUfoGLp  ...  Hotels and motels,Hotel and motel reservations...
3  AVwdtuoFByjofQCxr08w  ...  Hotel,Hotels and motels,Hotel and motel reser

In [44]:
def getCollabRec(u_id):
    

    userFlatRec =  sqlContext.createDataFrame(all_userRecoms.filter(col('uid') == u_id).rdd.flatMap(lambda p: p[1]))

    a = userFlatRec.alias("a")
    b = hotel_new_df.alias("b")
    
    return a.join(b, col("a.Id") == col("b.Id"), 'inner') \
             .select([ col('b.Id'),col('b.hotel_id'), col('a.rating'), col('b.name'),col('b.average_rating'),col('b.categories'),
                                                           col('b.latitude'),col('b.longitude')]) \
             .orderBy("rating", ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
import warnings
warnings.filterwarnings('ignore')

#print("---------------------------------\nEnter the user id: \n")
u_id= 6357
#u_id= input()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
getCollabRec(u_id).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     Id              hotel_id  ...   latitude   longitude
0    27  AVwckL-d_7pvs4fzwg58  ...  38.255101  -122.33346
1   430  AVweeQx7_7pvs4fzDTn7  ...   32.49094   -92.05335
2   350  AVweNSZkIN2L1WUf27Ye  ...    41.4512    -75.6368
3   115  AVwcrY5MkufWRAb5x1nm  ...   41.07905   -75.77149
4  1418  AWE2I5Nv3-Khe5l_fxSI  ...   31.88892  -102.33335
5  1268  AVweFB7p_7pvs4fz_18I  ...   34.17064   -97.16348
6   303  AVwd1Zo5ByjofQCxs7DO  ...  42.093964   -88.33668
7  1655  AVwclphkIN2L1WUfmqLw  ...    34.4445   -119.2532
8   961  AVwdYrkeIN2L1WUfvQ5Y  ...   32.67507    -97.0321
9  1758  AVwc5DdcIN2L1WUfqBcg  ...   32.52805   -93.69846

[10 rows x 8 columns]

In [53]:
result_df= getCollabRec(u_id).toPandas()
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

sdf = sqlContext.createDataFrame(result_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-53:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 7282



In [55]:
sdf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+------------------+--------------------+-----------------+--------------------+---------+----------+
|  Id|            hotel_id|            rating|                name|   average_rating|          categories| latitude| longitude|
+----+--------------------+------------------+--------------------+-----------------+--------------------+---------+----------+
|  27|AVwckL-d_7pvs4fzwg58|  4.70231294631958|    The Carneros Inn|              5.0|        Hotel,Hotels|38.255101|-122.33346|
| 430|AVweeQx7_7pvs4fzDTn7|2.6843972206115723|Best Western Airp...|              4.0|Hotels,Lodging,Ho...| 32.49094| -92.05335|
| 350|AVweNSZkIN2L1WUf27Ye|2.6791014671325684|Microtel Inn & Su...|4.916666666666667|Hotels and motels...|  41.4512|  -75.6368|
| 115|AVwcrY5MkufWRAb5x1nm|2.6437511444091797|Econo Lodge Inn &...|              3.5|Hotel,Hotels,Lodg...| 41.07905| -75.77149|
|1418|AWE2I5Nv3-Khe5l_fxSI| 2.562448263168335|  Hampton Inn Odessa|              5.0|Hotel,Hotels,Lodg..